In [2]:
# finlab 軟體包要上課才有喔～
from finlab_old.crawler import widget, date_range

# 讀入一些package
import sqlite3
import os
import requests
from io import StringIO
import pandas as pd

# 爬取資料
def crawl_legal_person(date):
    
    # 將時間物件變成字串：'20180102'
    datestr = date.strftime('%Y%m%d')
    
    # 下載三大法人資料
    try:
        r = requests.get('http://www.tse.com.tw/fund/T86?response=csv&date='+datestr+'&selectType=ALLBUT0999')#
    except:
        return None

    # 製作三大法人的DataFrame
    try:
        df = pd.read_csv(StringIO(r.text), header=1).dropna(how='all', axis=1).dropna(how='any')
    except:
        return None
    
    # 微調整（為了配合資料庫的格式）

    # 刪除逗點
    df = df.astype(str).apply(lambda s: s.str.replace(',',''))

    # 刪除「證券代號」中的「"」和「=」
    df['stock_id'] = df['證券代號'].str.replace('=','').str.replace('"','')

    # 刪除「證券代號」這個欄位
    df = df.drop(['證券代號'], axis=1)

    # 設定index
    df['date'] = date
    df = df.set_index(['stock_id', 'date'])
    
    # 將dataframe的型態轉成數字
    return df.apply(lambda s: pd.to_numeric(s, errors='coerce')).dropna(how='all', axis=1)

# 打開資料庫
conn = sqlite3.connect(os.path.join('data', 'data.db'))

# 開啟GUI介面
widget(conn, 'legal_person', crawl_legal_person, date_range)